In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg 

experiment:
  seed: '?' 
  torch_compile: False
  torch_mixed_precision: False
    
dataloader:
  batch_size: '?' 
  num_workers: '?'
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: '?'
  
train:
  epochs: '?' 
  eval_freq: '?'

# Used for additional data experiment.
data:
  train_kwargs:
    split: 'train'
  val_kwargs: 
    split: 'val'
  
loss_func: 
  _class: '?'
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml log_cfg

log:
  root: '?'
  checkpoint_freq: 20 
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True
    abs_area_estimation_error:
      _fn: ese.losses.area_estimation_error
      from_logits: True
      abs_diff: True

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg  

model:
  _class: ese.models.unet.UNet
  filters: '?'
  convs_per_block: '?' # Good default for UNets.

<IPython.core.display.Javascript object>

In [5]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [6]:
%%yaml aug_cfg

augmentations:
    spatial:
        max_translation: 5.0
        max_rotation: 5.0
        max_scaling: 1.1
        warp_integrations: 5
        warp_smoothing_range: [10, 20]
        warp_magnitude_range: [1, 3]
        affine_probability: 0.5
        warp_probability: 0.5
    visual:
        use_mask: False 
        added_noise_max_sigma: 0.05
        gamma_scaling_max: 0.8
        bias_field_probability: 0.5
        gamma_scaling_probability: 0.5
        added_noise_probability: 0.5

<IPython.core.display.Javascript object>

In [7]:
%%yaml experiment_cfg 

name: "Roads_RandomCrop500_big"

experiment:
    seed: 40
    seed_range: 3
    torch_compile: False  

data:
    _class: "ese.datasets.Roads"

dataloader:
  batch_size: 4
  num_workers: 3

model:
    convs_per_block: 4

optim:
    lr: 1.0e-4

loss_func:
    _class: ese.losses.SoftDiceLoss

<IPython.core.display.Javascript object>

In [8]:
from ese.analysis.analysis_utils.submit_utils import get_ese_training_configs 

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([model_cfg, log_cfg, callbacks_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, train_cfgs = get_ese_training_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [9]:
len(train_cfgs)

3

In [10]:
from pprint import pprint

pprint(train_cfgs[0])

Config({'experiment': {'seed': 40, 'torch_compile': False, 'torch_mixed_precision': False}, 'dataloader': {'batch_size': 4, 'num_workers': 3, 'pin_memory': True}, 'optim': {'_class': 'torch.optim.Adam', 'lr': 0.0001}, 'train': {'epochs': 3000, 'eval_freq': 30}, 'data': {'train_kwargs': {'split': 'train', 'transforms': [{'albumentations.RandomCrop': {'height': 500, 'width': 500}}]}, 'val_kwargs': {'split': 'val', 'transforms': [{'albumentations.RandomCrop': {'height': 500, 'width': 500}}]}, '_class': 'ese.datasets.Roads', 'version': 0.1, 'in_channels': 3, 'out_channels': 1}, 'loss_func': {'_class': 'ese.losses.SoftDiceLoss', 'from_logits': True, 'batch_reduction': 'mean'}, 'model': {'_class': 'ese.models.unet.UNet', 'filters': [64, 64, 64, 64, 64, 64, 64], 'convs_per_block': 4}, 'log': {'root': '/storage/vbutoi/scratch/ESE/training/10_08_24_Roads_RandomCrop500_big', 'checkpoint_freq': 20, 'metrics': {'dice_score': {'_fn': 'ionpy.metrics.dice_score', 'batch_reduction': 'mean', 'ignore_em

## Running Jobs

In [11]:
# ####### FOR DEBUGGIN
# from ese.experiment import run_ese_exp, CalibrationExperiment

# run_ese_exp(
#     config=train_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     run_name='debug',
#     show_examples=False,
#     track_wandb=False,
#     gpu='3',
# )

In [12]:
# FOR SUBMISSION
from ese.experiment import submit_ese_exps, CalibrationExperiment 

submit_ese_exps(
    group="training",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=train_cfgs,
    experiment_class=CalibrationExperiment,
    available_gpus=['0', '1', '2'],
    track_wandb=True
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Submitted job id: 1291806 on gpu: 0.
Submitted job id: 1291934 on gpu: 1.
Submitted job id: 1292092 on gpu: 2.
